<a href="https://colab.research.google.com/github/SATHWIK123423/Colab_notebooks/blob/main/ML_EVENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy
import torch
import torchvision
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import sys
import csv
from torchsummary import summary
from sklearn.metrics import confusion_matrix

In [2]:
cd=pd.read_csv('/content/customer_data.csv')
rd=pd.read_csv('/content/payment_data.csv')

In [ ]:
cd

,label,id,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,fea_9,fea_10,fea_11
0,1,54982665,5,1245.5,3,77000.0,2,15,5,109,5,151300,244.948974
1,0,59004779,4,1277.0,1,113000.0,2,8,-1,100,3,341759,207.173840
2,0,58990862,7,1298.0,1,110000.0,2,11,-1,101,5,72001,1.000000
3,1,58995168,7,1335.5,1,151000.0,2,11,5,110,3,60084,1.000000
4,0,54987320,7,NaN,2,59000.0,2,11,5,108,4,450081,197.403141
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,0,58988196,5,1289.0,1,173000.0,2,15,5,112,3,350702,200.000000
1121,0,58987926,5,NaN,2,50000.0,2,15,5,108,4,450000,169.000000
1122,0,58995381,7,1220.0,3,76000.0,2,11,2,90,5,71002,1.000000
1123,0,58998054,4,1250.0,3,137000.0,2,8,5,90,5,72000,1.000000


In [ ]:
rd

,id,OVD_t1,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
0,58987402,0,0,0,0,1,10,16500.0,04/12/2016,0.0,NaN,NaN
1,58995151,0,0,0,0,1,5,NaN,04/12/2016,588720.0,491100.0,NaN
2,58997200,0,0,0,0,2,5,NaN,04/12/2016,840000.0,700500.0,22/04/2016
3,54988608,0,0,0,0,3,10,37400.0,03/12/2016,8425.2,7520.0,25/04/2016
4,54987763,0,0,0,0,2,10,NaN,03/12/2016,15147.6,NaN,26/04/2016
...,...,...,...,...,...,...,...,...,...,...,...,...
8245,58995478,0,0,0,0,9,15,NaN,NaN,0.0,NaN,NaN
8246,54992408,0,0,0,0,1,2,NaN,NaN,0.0,NaN,NaN
8247,54988209,0,0,0,0,5,13,NaN,NaN,20654.4,33315.0,NaN
8248,54992408,0,0,0,0,1,2,NaN,NaN,0.0,NaN,NaN


In [3]:
d = (cd.merge(rd, left_on='id', right_on='id'))

In [4]:
d.shape

(8250, 24)

In [ ]:
d

,label,id,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,...,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
0,1,54982665,5,1245.5,3,77000.0,2,15,5,109,...,0,0,0,9,10,60500.0,31/01/2015,6657.6,18934.0,19/09/2015
1,1,54982665,5,1245.5,3,77000.0,2,15,5,109,...,0,0,0,18,10,NaN,28/04/2009,153792.0,149387.0,10/08/2015
2,1,54982665,5,1245.5,3,77000.0,2,15,5,109,...,0,0,0,1,5,NaN,30/03/2009,0.0,150500.0,NaN
3,1,54982665,5,1245.5,3,77000.0,2,15,5,109,...,2,26,11906,6,10,NaN,17/05/2007,0.0,46371.0,12/07/2013
4,0,59004779,4,1277.0,1,113000.0,2,8,-1,100,...,0,0,0,4,6,NaN,02/06/2016,15120.0,21500.0,19/04/2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8245,0,58998054,4,1250.0,3,137000.0,2,8,5,90,...,0,0,0,10,13,NaN,06/04/2010,0.0,35500.0,06/03/2011
8246,0,58998054,4,1250.0,3,137000.0,2,8,5,90,...,0,0,0,1,10,NaN,27/04/2008,-121.2,601.0,NaN
8247,0,54989781,4,1415.0,3,93000.0,2,8,5,113,...,0,0,0,12,2,NaN,01/12/2015,334130.4,2400500.0,12/01/2015
8248,0,54989781,4,1415.0,3,93000.0,2,8,5,113,...,0,0,0,7,1,NaN,18/07/2015,456098.4,406938.0,22/12/2015


In [5]:
result = d.dtypes
print(result)

label                int64
id                   int64
fea_1                int64
fea_2              float64
fea_3                int64
fea_4              float64
fea_5                int64
fea_6                int64
fea_7                int64
fea_8                int64
fea_9                int64
fea_10               int64
fea_11             float64
OVD_t1               int64
OVD_t2               int64
OVD_t3               int64
OVD_sum              int64
pay_normal           int64
prod_code            int64
prod_limit         float64
update_date         object
new_balance        float64
highest_balance    float64
report_date         object
dtype: object


In [ ]:
print(d.shape)

(8250, 24)

In [6]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
x.shape
y.shape

NameError: ignored

In [ ]:
d

,label,id,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,...,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
0,1,54982665,5,1245.5,3,77000.0,2,15,5,109,...,0,0,0,9,10,60500.0,31/01/2015,6657.6,18934.0,19/09/2015
1,1,54982665,5,1245.5,3,77000.0,2,15,5,109,...,0,0,0,18,10,NaN,28/04/2009,153792.0,149387.0,10/08/2015
2,1,54982665,5,1245.5,3,77000.0,2,15,5,109,...,0,0,0,1,5,NaN,30/03/2009,0.0,150500.0,NaN
3,1,54982665,5,1245.5,3,77000.0,2,15,5,109,...,2,26,11906,6,10,NaN,17/05/2007,0.0,46371.0,12/07/2013
4,0,59004779,4,1277.0,1,113000.0,2,8,-1,100,...,0,0,0,4,6,NaN,02/06/2016,15120.0,21500.0,19/04/2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8245,0,58998054,4,1250.0,3,137000.0,2,8,5,90,...,0,0,0,10,13,NaN,06/04/2010,0.0,35500.0,06/03/2011
8246,0,58998054,4,1250.0,3,137000.0,2,8,5,90,...,0,0,0,1,10,NaN,27/04/2008,-121.2,601.0,NaN
8247,0,54989781,4,1415.0,3,93000.0,2,8,5,113,...,0,0,0,12,2,NaN,01/12/2015,334130.4,2400500.0,12/01/2015
8248,0,54989781,4,1415.0,3,93000.0,2,8,5,113,...,0,0,0,7,1,NaN,18/07/2015,456098.4,406938.0,22/12/2015


In [7]:
class data(Dataset):
  def __init__(self):
    trdata=d
    xy=trdata.iloc[:,:].values
    labelen_xy=LabelEncoder()
    xy[:,20]=labelen_xy.fit_transform(xy[:,20])
    xy[:,23]=labelen_xy.fit_transform(xy[:,23])
    xy=xy.astype('float32')
    m=MinMaxScaler()
    xy=m.fit_transform(xy)
    self.x=torch.from_numpy(xy[:,2:24])
    self.y=torch.from_numpy(xy[:,[0]])
    self.y=self.y.long()
    t_o=torch.zeros(len(self.y),2)
    self.y=t_o.scatter_(1,self.y,1.0)
    self.XY=pd.DataFrame(xy)
    self.n_samples=xy.shape[0]

  def __getitem__(self,index):
    return self.x[index],self.y[index]
  def __len__(self):
    return self.n_samples
dataset=data()
first_data=dataset[0]
features,labels=first_data
print(features,labels)


tensor([6.6667e-01, 3.5391e-01, 1.0000e+00, 5.2321e-02, 1.0000e+00, 9.2308e-01,
        5.4545e-01, 8.8235e-01, 1.0000e+00, 1.5473e-01, 3.4548e-01, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 2.5000e-01, 3.7037e-01, 9.1665e-02,
        9.8520e-01, 2.8766e-04, 1.0241e-04, 5.6713e-01]) tensor([0., 1.])


In [ ]:
print(len(dataset))
print(len(dataset[0][0]))

8250
22


In [ ]:
x2=dataset[0][0][8]

In [8]:
for i in range(0,8250):
  for j in range(0,22):
    if(dataset[i][0][j]>=0 and dataset[i][0][j]<=1):
      continue
    else:
      dataset[i][0][j]=0

In [ ]:
dataset[0][0][8]

tensor(0.)

In [9]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.15, random_state=42)
train_loader=DataLoader(dataset=train_dataset,shuffle=True)
test_loader=DataLoader(dataset=test_dataset,shuffle=True)


In [10]:
train_loader=DataLoader(dataset=train_dataset,shuffle=True,batch_size=64)
test_loader=DataLoader(dataset=test_dataset,shuffle=True,batch_size=63)

In [11]:
from torch.distributions import Normal
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
from torchvision.utils import make_grid
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision.utils import save_image


In [12]:
class mainm(nn.Module):
  def __init__(self):
    super().__init__()
    self.sath=nn.Sequential(nn.Linear(22,30),
                         nn.ReLU(),
                         nn.Linear(30,40),
                         nn.ReLU(),
                         nn.Linear(40,30),
                         nn.ReLU(),
                         nn.Linear(30,30),
                         nn.ReLU(),
                         nn.Linear(30,30),
                         nn.ReLU(),
                         nn.Linear(30,30),
                         nn.ReLU(),
                         nn.Linear(30,10),
                         nn.ReLU(),
                         nn.Linear(10,5),
                         nn.ReLU(),
                         nn.Linear(5,2),
                         nn.Softmax() 
                                        )
   
  def forward(self,x):
    out=self.sath(x)
    return out

In [13]:
mmodel=mainm().to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(mmodel.parameters(),lr=0.001,weight_decay=0.000001)

In [ ]:
mmodel

mainm(
  (sath): Sequential(
    (0): Linear(in_features=22, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=30, bias=True)
    (5): ReLU()
    (6): Linear(in_features=30, out_features=30, bias=True)
    (7): ReLU()
    (8): Linear(in_features=30, out_features=30, bias=True)
    (9): ReLU()
    (10): Linear(in_features=30, out_features=30, bias=True)
    (11): ReLU()
    (12): Linear(in_features=30, out_features=10, bias=True)
    (13): ReLU()
    (14): Linear(in_features=10, out_features=5, bias=True)
    (15): ReLU()
    (16): Linear(in_features=5, out_features=2, bias=True)
    (17): Softmax(dim=None)
  )
)

In [14]:
def accuracy(outputs, labels):
    preds = torch.max(outputs,dim=1)
    L=torch.max(labels,dim=1)
    return torch.tensor(torch.sum(preds[1] == L[1]).item() / len(preds[1]))*100

In [15]:
from sklearn.metrics import confusion_matrix

In [16]:
nepoch=30
output=[]
plottr=[]
plotts=[]
losstr=[]
lossts=[]
lss=0
acc=0
for epoch in range (nepoch):
  acc=0
  lss=0
  i=0
  for d,w in train_loader:
    i+=1
    d = d.to(device)
    w = w.to(device)
    o=mmodel(d)
    #print(o)
    loss=criterion(o,w)
    lss+=loss
    acc+=accuracy(o,w)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  lss=lss/i
  acc=acc/i
  plottr.append(acc)
  losstr.append(lss)
  if((epoch+1)%5==0):
    print(f'Epoch:{epoch},Loss:{loss.item():.4f},TRAIN_ACCURACY:{acc}% lenght={len(w)}')
  i=0
  acc=0
  lss=0
  for d,w in test_loader:
    i+=1
    d = d.to(device)
    w = w.to(device)   
    o=mmodel(d)
    #print(o)
    loss=criterion(o,w)
    lss+=loss
    acc+=accuracy(o,w)
  lss=lss/i
  acc=acc/i
  lossts.append(lss)
  plotts.append(acc)
  if((epoch+1)%5==0):
    print(f'Epoch:{epoch},Loss:{loss.item():.4f},VAL_ACCURACY:{acc}% ')

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch:4,Loss:0.3688,TRAIN_ACCURACY:83.1029052734375% lenght=36
Epoch:4,Loss:0.5084,VAL_ACCURACY:83.86565399169922% 
Epoch:9,Loss:0.6188,TRAIN_ACCURACY:83.00347137451172% lenght=36
Epoch:9,Loss:0.4108,VAL_ACCURACY:84.0360107421875% 
Epoch:14,Loss:0.4799,TRAIN_ACCURACY:83.05870819091797% lenght=36
Epoch:14,Loss:0.4840,VAL_ACCURACY:83.90824890136719% 
Epoch:19,Loss:0.4799,TRAIN_ACCURACY:83.05870819091797% lenght=36
Epoch:19,Loss:0.4840,VAL_ACCURACY:83.90824890136719% 
Epoch:24,Loss:0.4522,TRAIN_ACCURACY:83.06976318359375% lenght=36
Epoch:24,Loss:0.5328,VAL_ACCURACY:83.82307434082031% 
Epoch:29,Loss:0.3688,TRAIN_ACCURACY:83.1029052734375% lenght=36
Epoch:29,Loss:0.3864,VAL_ACCURACY:84.07859802246094% 


In [17]:
dataset[0]

(tensor([6.6667e-01, 3.5391e-01, 1.0000e+00, 5.2321e-02, 1.0000e+00, 9.2308e-01,
         5.4545e-01, 8.8235e-01, 1.0000e+00, 1.5473e-01, 3.4548e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 2.5000e-01, 3.7037e-01, 9.1665e-02,
         9.8520e-01, 2.8766e-04, 1.0241e-04, 5.6713e-01]), tensor([0., 1.]))

In [18]:
d1=dataset[0][0].to(device)
l=mmodel(d1)
print(l)

tensor([1.0000e+00, 4.2782e-09], device='cuda:0', grad_fn=<SoftmaxBackward0>)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [22]:
t1=[]
t2=[]
i=0
for d,w in test_dataset:
  i+=1
  if(i>100):
    break
  o=d.to(device)
  l=mmodel(o)
  if(l[0]>0.5):
    t1.append(0)
  else:
    t1.append(1)
  if(w[0]>0.5):
    t2.append(0)
  else:
    t2.append(1)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [23]:
confusion_matrix(t1,t2)

array([[83, 17],
       [ 0,  0]])

In [25]:
 from sklearn.metrics import f1_score
 f1_score(t1, t2, average='micro')

0.83